<a href="https://colab.research.google.com/github/bmeelnlga/myportfolio/blob/main/%E9%81%B8%E6%93%87%E6%AC%8ACRR%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
#歐式選擇權參數設定
#期初股價
S0 = 100
#履約價格
K = 100
#契約期間
T = 1
#無風險利率
r = 0.0165
#股票報酬率標準差
sigma = np.sqrt(1.25)
#考量期數
M = 3
#每期時段
dt = T / M
#折現因子
df = np.exp(-r * dt)
#股價上漲因子
u = np.exp(sigma * np.sqrt(dt))
#股價下跌因子
d = 1 / u
#風險中立下股價上漲對應的機率
p = (np.exp(r * dt) - d) / (u - d)

In [ ]:
def binomial(x, y):  # x = -1 for put, 1 for call # y = 0 for European, 1 for American
  S = np.zeros((M + 1,M + 1))  #將契約期間分為M期 故包含期初0 共有(M + 1)個時間點 因此建立一個(M + 1) * (M + 1)全為0的陣列 即將股價走勢設定於((M + 1) * (M + 1))的矩陣 值得注意的是Python陣列中 行列的起始值均為0 故行列均為 0,1,...,M 共(M + 1)
  S [0 , 0] = S0  #期初股價起始點
  for j in range(1, M + 1):  #j矩陣的column(行)表示第j期 因為期初股價已經設定S [0 , 0] = S0 故股價樹應建構第1期至第M期 其中range(1 , M + 1) python的顯示方式為由 1,2,3,...,M
    for i in range(0, j+1):  #i矩陣的row(列)表示股價下跌的次數 以第2期為例 股價路徑包括 (u^2)SO (ud)S0 = (du)S0 (d^2)SO 3種可能路徑 其中range(0, 2+1)表示0(0次下跌),1(1次下跌),2(2次下跌)
      S[i , j] = S[0 , 0] * (u ** (j-i)) * (d ** i)

  iv = np.zeros((M + 1,M + 1))  #intrinsic value(iv)內涵價值或稱履約價值
  for j in range(0, M + 1):
    for i in range(0, j+1):
      iv[i , j] = max(x * (S[i,j] - K), 0)  #x = -1 與 1 分別表示賣權與買權的內涵價值

  pv = np.zeros((M + 1,M + 1))
  pv[:, M] = iv[:, M]  #表示在第M期(期末)的現值 其中符號: 表示由第0列至第M列 等於內涵價值
  for j in range(M-1, -1, -1):  #range(M-1, -1, -1)表示由後往前 第一項參數表示第M - 1期 第二項參數-1 表示往前至第0期(-1的前一項為0) 第三項參數-1表示每一次移動為往前一期
    for i in range(0, j+1):
      pv[i , j] =max( df * ((p * pv[i , j + 1]) + ((1 - p) * (pv[i + 1 , j + 1]))), y * iv[i , j])
  return S, pv

In [ ]:
binomial(-1,1)

(array([[100.        , 190.69349681, 363.64009726, 693.43801728],
        [  0.        ,  52.4401732 , 100.        , 190.69349681],
        [  0.        ,   0.        ,  27.49971765,  52.4401732 ],
        [  0.        ,   0.        ,   0.        ,  14.42089956]]),
 array([[44.32492509, 19.99705612,  0.        ,  0.        ],
        [ 0.        , 57.68433474, 30.83920436,  0.        ],
        [ 0.        ,  0.        , 72.50028235, 47.5598268 ],
        [ 0.        ,  0.        ,  0.        , 85.57910044]]))

In [ ]:
!pip install yfinance
import yfinance as yf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
data = yf.download("2330.TW", start="2022-01-01", end="2022-11-21")["Close"] #股價僅會抓至11-18
print(data.head(5),data.tail(5))  #檢視前五筆與後五筆日股價

returns = data.pct_change()  #日報酬率
print(returns.head(5),returns.tail(5))  #檢視前五筆與後五筆日報酬率

print(returns.describe())  #檢視日報酬率敘述統計表

mean_daily = returns.mean()     #日平均報酬率
print(mean_daily)
sigma_daily = returns.std()     #日報酬率標準差
print(sigma_daily)

#把日報酬率標準差年化 *根號(交易天數)
sigma_yearly = sigma_daily * np.sqrt(returns.count()) #假設每日報酬率相互獨立且有相同分配 年化變異數為日變異數乘以交易天數 年標準差即為日標準差乘以根號(交易天數)
print(sigma_yearly)



[*********************100%***********************]  1 of 1 completed
Date
2022-01-03    631.0
2022-01-04    656.0
2022-01-05    650.0
2022-01-06    644.0
2022-01-07    634.0
Name: Close, dtype: float64 Date
2022-11-14    445.0
2022-11-15    480.0
2022-11-16    487.0
2022-11-17    485.0
2022-11-18    487.0
Name: Close, dtype: float64
Date
2022-01-03         NaN
2022-01-04    0.039620
2022-01-05   -0.009146
2022-01-06   -0.009231
2022-01-07   -0.015528
Name: Close, dtype: float64 Date
2022-11-14    0.007928
2022-11-15    0.078652
2022-11-16    0.014583
2022-11-17   -0.004107
2022-11-18    0.004124
Name: Close, dtype: float64
count    215.000000
mean      -0.000986
std        0.020984
min       -0.083333
25%       -0.014476
50%       -0.001996
75%        0.012064
max        0.083436
Name: Close, dtype: float64
-0.0009859085516816827
0.02098358574248979
0.3076797640106845
